XGBoost model for predicting filtered shear stress and Time to next event (TTF)

In [13]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
import statistics
from sklearn import multioutput
from sklearn.model_selection import train_test_split, ParameterGrid
from xgboost import XGBRegressor, plot_importance
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import h5py

In [14]:


def create_timesteps(data, n_steps):
	x = []
	y = []
	for i in range(len(data)-1):
		end_ix = i + n_steps
		if end_ix > len(data)-1:
			break
		x1, y1 = data[i:end_ix, :-2], data[end_ix, -2:]  #last two columns are the target variables
		x.append(x1)
		y.append(y1)
	return np.array(x), np.array(y)
 

In [15]:
# Load the data

data = loadmat('/content/drive/My Drive/Colab Notebooks/Labquake_prediction/MLpreprocessed_code/data/p5270_ML_Master.mat')

m_t = pd.DataFrame({'Time':data['Time'].ravel().round(2), 'SS':data['SS'].ravel(), 'TTF':data['TTF'].ravel()})
a_t = pd.DataFrame({'Time':data['LocalAcTime'].ravel().round(2), 'V_filt':data['V_filt'].ravel(), 'Vel_pc':data['Vel_pc'].ravel()})  
df1 = a_t.merge(m_t, on='Time')

data2 = loadmat('/content/drive/My Drive/Colab Notebooks/Labquake_prediction/MLpreprocessed_code/data/p5270_run1_pp_wAmp.mat')
df2 = pd.DataFrame({'maxFreqI_filt':data2['maxFreqI_filt'][3787:136186].ravel(), 'freqQAmpI_filt':data2['freqQAmpI_filt'][3787:136186].ravel(),
                    'freqQAmpI_filt_pc':data2['freqQAmpI_filt_pc'].ravel()})

df = pd.concat([df1, df2], axis=1)

df = df[['freqQAmpI_filt', 'freqQAmpI_filt_pc', 'V_filt', 'Vel_pc', 'maxFreqI_filt', 'SS', 'TTF']]
print("Input data:\n", df)


Input data:
         freqQAmpI_filt  freqQAmpI_filt_pc  ...        SS  TTF
0         31438.277060                0.0  ...  5.656166  0.0
1         31421.341616                0.0  ...  5.657157  0.0
2         31409.798761                0.0  ...  5.658317  0.0
3         31398.815589                0.0  ...  5.659499  0.0
4         31395.584981                0.0  ...  5.660404  0.0
...                ...                ...  ...       ...  ...
132394    31041.955345                0.0  ...  5.471059  0.0
132395    31035.059531                0.0  ...  5.474545  0.0
132396    31020.001291                0.0  ...  5.478286  0.0
132397    31012.016780                0.0  ...  5.481751  0.0
132398    31010.874482                0.0  ...  5.484813  0.0

[132399 rows x 7 columns]


In [16]:
# Preprocessing

arr = df.to_numpy()
n_steps = 300
xdf, ydf = create_timesteps(arr, n_steps)

print('Features shape, X = ', np.shape(xdf))
print('Target shape, Y = ', np.shape(ydf))

# Reshape features from 3D to 2D (for input layer)

in_dim = xdf.shape[1]*xdf.shape[2]
xdf = xdf.reshape((xdf.shape[0], in_dim))
print('After reshaping, X = ', np.shape(xdf))

Features shape, X =  (132099, 300, 5)
Target shape, Y =  (132099, 2)
After reshaping, X =  (132099, 1500)


In [17]:
# Split into train-val-test
x_train, x_test, y_train, y_test = train_test_split(xdf, ydf, test_size=0.2, shuffle=False)
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.1, shuffle=False)


In [ ]:
# Parameter tuning

params = {
        'learning_rate': [0.01, 0.1, 0.5],
        'n_estimators': [200, 600, 1000],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 5, 7, 9],
        'objective':['reg:squarederror']
        }

best_score = 0

for g in ParameterGrid(params):
  model = XGBRegressor()
  model.set_params(**g)
  model = multioutput.MultiOutputRegressor(model)
  model.fit(X_train, Y_train)
  y_predVal = model.predict(X_val)
  val_r2 = r2_score(Y_val, y_predVal)
  if val_r2 > best_score:
    best_score = val_r2
    best_grid = g
#print(best_grid)


In [19]:
# Train the model
#model = XGBRegressor(colsample_bytree=1, learning_rate=0.1, max_depth=5, n_estimators=600, objective='reg:squarederror')
model = XGBRegressor()
model.set_params(**best_grid)
model = multioutput.MultiOutputRegressor(model)
model.fit(X_train, Y_train)


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:614: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase memory consumption")


MultiOutputRegressor(estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            importance_type='gain',
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=5, min_child_weight=1,
                                            missing=None, n_estimators=600,
                                            n_jobs=1, nthread=None,
                                            objective='reg:squarederror',
                                            random_state=0, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                            seed=None, silent=None, subsample=1,
                                            

In [20]:
# Evaluate the model

## Training
y_predTrain = pd.DataFrame(model.predict(X_train))
y_predTrain[1] = np.clip(y_predTrain[1], a_min=0, a_max=None)

ss_train_r2 = r2_score(Y_train[:,0], y_predTrain[0])
ss_train_rmse = np.sqrt(mean_squared_error(Y_train[:,0], y_predTrain[0]))

ttf_train_r2 = r2_score(Y_train[:,1], y_predTrain[1])
ttf_train_rmse = np.sqrt(mean_squared_error(Y_train[:,1], y_predTrain[1]))


## Val
y_predVal = pd.DataFrame(model.predict(X_val))
y_predVal[1] = np.clip(y_predVal[1], a_min=0, a_max=None)

ss_val_r2 = r2_score(Y_val[:,0], y_predVal[0])
ss_val_rmse = np.sqrt(mean_squared_error(Y_val[:,0], y_predVal[0]))

ttf_val_r2 = r2_score(Y_val[:,1], y_predVal[1])
ttf_val_rmse = np.sqrt(mean_squared_error(Y_val[:,1], y_predVal[1]))

## Testing
y_predTest = pd.DataFrame(model.predict(x_test))
y_predTest[1] = np.clip(y_predTest[1], a_min=0, a_max=None)

ss_test_r2 = r2_score(y_test[:,0], y_predTest[0])
ss_test_rmse = np.sqrt(mean_squared_error(y_test[:,0], y_predTest[0]))

ttf_test_r2 = r2_score(y_test[:,1], y_predTest[1])
ttf_test_rmse = np.sqrt(mean_squared_error(y_test[:,1], y_predTest[1]))


print('Shear Stress')
print('R2 score:', ss_train_r2, ss_val_r2, ss_test_r2, '\nRMSE:', ss_train_rmse, ss_val_rmse, ss_test_rmse)
print('\nTTF')
print('R2 score:', ttf_train_r2, ttf_val_r2, ttf_test_r2, '\nRMSE:', ttf_train_rmse, ttf_val_rmse, ttf_test_rmse)


Shear Stress
R2 score: 0.9969209150802087 0.9591462889799628 0.8882204021216867 
RMSE: 0.007133119102595848 0.029823592893577253 0.04830921658847603

TTF
R2 score: 0.9934169039503822 0.9130198332691188 0.8317145278233581 
RMSE: 0.08504539075208456 0.3399725651345053 0.47107362426972393


In [ ]:
# Save the predictions
'''
hf = h5py.File('/content/drive/My Drive/Colab Notebooks/Labquake_prediction/MLpreprocessed_code/predictions/xgb_mo.h5', 'w')
g1 = hf.create_group('ss')
g1.create_dataset('y_predTrain', data=y_predTrain[0])
g1.create_dataset('y_predTest', data=y_predTest[0])

g2 = hf.create_group('ttf')
g2.create_dataset('y_predTrain', data=y_predTrain[1])
g2.create_dataset('y_predTest', data=y_predTest[1])
hf.close()
'''

In [ ]:
# Overall plot

ttime = df1['Time'][n_steps:]
traintime, testtime = train_test_split(ttime, test_size=0.3, shuffle=False)

## SS plot
fig = plt.figure(1, figsize=(20,6))
plt.plot(ttime, ydf['SS'])
plt.plot(traintime, y_predTrain[0])
plt.plot(testtime, y_predTest[0], 'tab:red')
plt.xlabel('Time (s)')
plt.ylabel('Shear Stress (MPa)')
plt.text(ttime.iloc[0], 5.3, 'Test R2 Score: %0.5f' %(ss_test_r2), style='italic', bbox=dict(facecolor='red', alpha=0.5, pad=10))
plt.legend(['Ground truth', 'Train Prediction', 'Test Prediction'])
plt.title('Shear Stress prediction using multioutput XGBoost model')

## TTF plot
fig = plt.figure(2, figsize=(20,6))
plt.plot(ttime, ydf['TTF'])
plt.plot(traintime, y_predTrain[1])
plt.plot(testtime, y_predTest[1], 'tab:red')
plt.xlabel('Time (s)')
plt.ylabel('TTF (s)')
plt.text(ttime.iloc[10000], 4.2, 'Test R2 Score: %0.5f' %(ttf_test_r2), style='italic', bbox=dict(facecolor='red', alpha=0.5, pad=10))
plt.legend(['Ground truth', 'Train Prediction', 'Test Prediction'])
plt.title('Time to Failure prediction using multioutput XGBoost model')



In [ ]:
# Detailed plot (test data)

n = 4000
st_i = 3200

## SS
fig = plt.figure(6, figsize=(20,6))
plt.plot(testtime[st_i:st_i+n], y_test['SS'][st_i:st_i+n])
plt.plot(testtime[st_i:st_i+n], y_predTest[0][st_i:st_i+n], 'r')
plt.xlabel('Time (s)')
plt.ylabel('Shear Stress (MPa)')
plt.legend(['Ground truth', 'Predicted'])
plt.title('Detailed View, Testing Data')

## TTF
fig = plt.figure(7, figsize=(20,6))
plt.plot(testtime[st_i:st_i+n], y_test['TTF'][st_i:st_i+n])
plt.plot(testtime[st_i:st_i+n], y_predTest[1][st_i:st_i+n], 'r')
plt.xlabel('Time (s)')
plt.ylabel('Shear Stress (MPa)')
plt.legend(['Ground truth', 'Predicted'])
plt.title('Detailed View, Testing Data')